In [13]:
import os
import numpy as np
import pandas as pd
import statsmodels.api as sm
from ISLP import load_data
from sklearn.utils import resample
import matplotlib.pyplot as plt

In [4]:
np.random.seed(42)
default = load_data('default')
default = pd.DataFrame(default)

# fit the logistic regression model
x = default[['income','balance']]
X_train = sm.add_constant(x)

default['default01'] = default['default'].map({'Yes':1, 'No': 0})
y = default['default01']
model_logistic = sm.GLM( y, X_train, family=sm.families.Binomial()).fit()
model_logistic.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:              default01   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9997
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -789.48
Date:                Tue, 25 Feb 2025   Deviance:                       1579.0
Time:                        14:54:42   Pearson chi2:                 6.95e+03
No. Iterations:                     9   Pseudo R-squ. (CS):             0.1256
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -11.5405      0.435    -26.544      0.000     -12.393     -10.688
income      2.081e-05   4.99e-06      4.174      0.000     1.1e-05    3.06e-05
balance        0.0056      0.000     24.835      0.000       0.005       0.006
==============================================================================
"""

### **(B&C)**  
Write a function `boot_fn()` that takes as input the Default data set as well as an index of the observations, and that outputs the coefficient estimates for income and balance in the multiple logistic regression model.

In [10]:
def boot_fn(data, index, predictors,response):
    boot_sample = data.iloc[index]  
    # predictors and response
    X_train = sm.add_constant(boot_sample[predictors])
    y = boot_sample[response]
    
    # logistic regression model
    model = sm.GLM(y, X_train, family=sm.families.Binomial()).fit()
    
    # Return the coefficients of income and balance
    return model.params[predictors].values


# iterations
n_bootstraps = 1000  
predictors = ['income', 'balance']
response = ['default01']
# Store estimates
boot_coefs = np.zeros((n_bootstraps, len(predictors)))  # Store values for income and balance

# Perform bootstrap sampling
for i in range(n_bootstraps):
    indices = resample(np.arange(len(default)), replace=True)  # Sample indices with replacement
    boot_coefs[i, :] = boot_fn(default, indices, predictors, response)  # Compute coefficients

# standard errors from the bootstrap estimates
boot_se = boot_coefs.std(axis=0)

# Print results
print(f"Bootstrap SE for income: {boot_se[0]:.4f}")
print(f"Bootstrap SE for balance: {boot_se[1]:.4f}")


Bootstrap Standard Error for Income: 0.0000
Bootstrap Standard Error for Balance: 0.0002
